In [1]:
import copy
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import datetime
import time
import joblib
from datetime import timedelta, date
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import os
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from statsmodels.stats.stattools import durbin_watson
from statsmodels.graphics.api import qqplot
from sklearn.decomposition import PCA

from sklearn.metrics import mean_squared_error

import seaborn as sns;

import warnings
warnings.filterwarnings("ignore")

sns.set_theme()

import errno

from numpy import array

import tensorflow as tf
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

2022-11-05 11:07:13.369800: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# PATHS TO DATA OR RESULTS

data_path = "./Year data P//"
results_path = "./results//"


In [3]:
df = pd.read_csv(data_path + 'Germany//Germany Data.csv')
df = df.loc[df['year'] >= 1980]
df = df.drop(['countrycode','country','currency_unit','year'],axis = 1)
year = pd.period_range('1980', '2019', freq='Y')
df.index = year

In [4]:
def rmse(pred, actual):
    return np.sqrt(((pred - actual) ** 2).mean())


# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences.iloc[i:end_ix, :], sequences.iloc[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def reshape_sequence(sequence, n_steps_in):
    X = list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in

        # check if we are beyond the sequence
        if end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix,:-1]
        X.append(seq_x)

    return np.array(X)

In [5]:
from sklearn.model_selection import ShuffleSplit
rs = ShuffleSplit(n_splits=1, test_size=.25, random_state=0)

def shuffled_test_train(n_inp, n_out, sequence):
    batches = []
    for i in range(len(sequence)):
        if (i + n_inp + n_out > len(sequence)):
            break
        batches.append(sequence[i:i + n_inp + n_out])
  
    train = []
    test = []
    for train_index, test_index in rs.split(batches):
        print("TRAIN:", train_index, "TEST:", test_index)
    for i in train_index:
        train.append(batches[i])
    for i in test_index:
        test.append(batches[i])
    
    train = np.array(train)
    trainX = train[:,:n_inp,:-1]
    trainY = train[:,n_inp:,:-1]
    test = np.array(test)
    testX = test[:,:n_inp,:-1]
    testY = test[:,n_inp:,:-1]

    return train,test,trainX,trainY,testX,testY

In [6]:
x,y = split_sequences(df,5,3)

In [7]:
train,test,x_train,y_train,x_test,y_test = shuffled_test_train(5,3,df)

TRAIN: [32 29 23 15  8 13 30  5 14 28 22  1 12  6  4 18 21 19  9  7 31 27  3  0] TEST: [11 20 24 17  2 25 26 16 10]


In [8]:
x_train.shape

(24, 5, 20)

In [9]:
n_steps_in = 5
n_steps_out = 3
Num_Exp = 50
Hidden = 10
Epochs= 1000

In [10]:
Xscaler =MinMaxScaler()
df.iloc[:,0:-1]=Xscaler.fit_transform(df.values[:,0:-1])

Yscaler = MinMaxScaler()
Yscaled=Yscaler.fit_transform(df.values[:,-1].reshape(-1,1))
df.iloc[:,-1] = Yscaled.reshape(-1,)

In [11]:
df.shape

(40, 21)

In [12]:
n_features = x_train.shape[2]
x_features = df.shape[1]-1
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], n_features))
print(x_train.shape)
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], n_features))
print(x_test.shape)
x = reshape_sequence(df.values,n_steps_in)
x = x.reshape((x.shape[0], x.shape[1], x_features))

(24, 5, 20)
(9, 5, 20)


In [13]:
train_acc = np.zeros(Num_Exp)
test_acc = np.zeros(Num_Exp)
Step_RMSE = np.zeros([Num_Exp, n_steps_out])



model = Sequential()
model.add(LSTM(300, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(300, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()
Best_RMSE = 1000  # Assigning a large number

start_time = time.time()
for run in range(Num_Exp):
    print("Experiment", run + 1, "in progress")
    # fit model
    model.fit(x_train, y_train, epochs=Epochs, batch_size=64, verbose=0, shuffle=False)
    y_predicttrain = model.predict(x_train)
    y_predicttest = model.predict(x_test)
    y_predict = model.predict(x)
    train_acc[run] = rmse(y_predicttrain, y_train)
    test_acc[run] = rmse(y_predicttest, y_test)
    if test_acc[run] < Best_RMSE:
        Best_RMSE = test_acc[run]
        Best_Predict_Test = y_predicttest
        Best_Predict = y_predict
    for j in range(n_steps_out):
        Step_RMSE[run][j] = rmse(y_predicttest[:, j], y_test[:, j])
#print('hh',Best_Predict)
y = x[[-5,-4,-3, -2, -1], :]
y = y.reshape((y.shape[0], y.shape[1], n_features))
y1_predict=model.predict(y)
y2_predict=model.predict(y1_predict)
Best_Predict=np.row_stack((Best_Predict,y1_predict,y2_predict))
print(Best_Predict.shape)

print("Total time for", Num_Exp, "experiments", time.time() - start_time)

2022-11-05 11:09:13.703576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 300)               385200    
                                                                 
 repeat_vector (RepeatVector  (None, 3, 300)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 3, 300)            721200    
                                                                 
 time_distributed (TimeDistr  (None, 3, 20)            6020      
 ibuted)                                                         
                                                                 
Total params: 1,112,420
Trainable params: 1,112,420
Non-trainable params: 0
_________________________________________________________________
Experiment 1 in progress
2/2 [================

2/2 [==============================] - 0s 4ms/step
Experiment 41 in progress
2/2 [==============================] - 0s 3ms/step
Experiment 42 in progress
2/2 [==============================] - 0s 3ms/step
Experiment 43 in progress
2/2 [==============================] - 0s 4ms/step
Experiment 44 in progress
2/2 [==============================] - 0s 3ms/step
Experiment 45 in progress
2/2 [==============================] - 0s 3ms/step
Experiment 46 in progress
2/2 [==============================] - 0s 4ms/step
Experiment 47 in progress
2/2 [==============================] - 0s 4ms/step
Experiment 48 in progress
2/2 [==============================] - 0s 5ms/step
Experiment 49 in progress
2/2 [==============================] - 0s 3ms/step
Experiment 50 in progress
1/1 [==============================] - 0s 265ms/step
(46, 3, 20)
Total time for 50 experiments 744.6178889274597


In [14]:
Best_Predict[:,0,0]

array([2.8228996 , 3.7842758 , 4.54978   , 4.925309  , 4.891593  ,
       4.7375298 , 3.2700038 , 3.5078115 , 3.6930528 , 2.9073634 ,
       2.1964636 , 3.3277597 , 3.557268  , 4.0944223 , 4.19708   ,
       2.6434407 , 3.6211953 , 3.8765018 , 4.17966   , 5.2707043 ,
       5.9924765 , 6.3918295 , 5.8953075 , 2.5988884 , 3.3351974 ,
       3.9234977 , 3.5246677 , 3.0294397 , 2.2171962 , 3.8335655 ,
       4.039049  , 3.3087792 , 3.9457755 , 3.408879  , 3.5326    ,
       4.2169943 , 2.258063  , 3.4030955 , 2.748075  , 3.156876  ,
       3.77882   , 0.17460501, 1.9698915 , 0.5150583 , 1.8113594 ,
       1.5200684 ], dtype=float32)

In [15]:
Best_Predict  = Best_Predict[:,0,0]
#np.row_stack((Best_Predict[:-5,0],Best_Predict[-5,:]))

Best_Predict = Yscaler.inverse_transform(Best_Predict.reshape(-1,1))  

In [16]:
df.iloc[:,0]

1980    0.648229
1981    0.569673
1982    0.484299
1983    0.664119
1984    0.779035
1985    0.733192
1986    0.730059
1987    0.648857
1988    0.859287
1989    0.877663
1990    1.000000
1991    0.988369
1992    0.696168
1993    0.431393
1994    0.739960
1995    0.661535
1996    0.595242
1997    0.683908
1998    0.705358
1999    0.693190
2000    0.786064
2001    0.674935
2002    0.502328
2003    0.456428
2004    0.628911
2005    0.587039
2006    0.869279
2007    0.793276
2008    0.608428
2009    0.000000
2010    0.903281
2011    0.878469
2012    0.559663
2013    0.560212
2014    0.723376
2015    0.656593
2016    0.724529
2017    0.765684
2018    0.619961
2019    0.616741
Freq: A-DEC, Name: GDP, dtype: float64

In [17]:
Best_Predict

array([[ 2.5186188e+01],
       [ 3.5703651e+01],
       [ 4.4078274e+01],
       [ 4.8186565e+01],
       [ 4.7817711e+01],
       [ 4.6132259e+01],
       [ 3.0077511e+01],
       [ 3.2679131e+01],
       [ 3.4705669e+01],
       [ 2.6110222e+01],
       [ 1.8332973e+01],
       [ 3.0709360e+01],
       [ 3.3220181e+01],
       [ 3.9096657e+01],
       [ 4.0219730e+01],
       [ 2.3222906e+01],
       [ 3.3919548e+01],
       [ 3.6712605e+01],
       [ 4.0029156e+01],
       [ 5.1965191e+01],
       [ 5.9861385e+01],
       [ 6.4230309e+01],
       [ 5.8798355e+01],
       [ 2.2735502e+01],
       [ 3.0790730e+01],
       [ 3.7226738e+01],
       [ 3.2863537e+01],
       [ 2.7445738e+01],
       [ 1.8559788e+01],
       [ 3.6242878e+01],
       [ 3.8490871e+01],
       [ 3.0501715e+01],
       [ 3.7470459e+01],
       [ 3.1596807e+01],
       [ 3.2950314e+01],
       [ 4.0437592e+01],
       [ 1.9006872e+01],
       [ 3.1533535e+01],
       [ 2.4367605e+01],
       [ 2.8839893e+01],


In [18]:
print('test',test_acc.mean(),'train',train_acc.mean())

test 0.5801899990849698 train 0.002092059173423438
